In [ ]:
#Imports
import requests
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter, column_index_from_string
from openpyxl.styles import Alignment, Font, PatternFill, Border, Side, Color
from openpyxl.worksheet.hyperlink import Hyperlink
from openpyxl.formatting.rule import FormulaRule

In [ ]:
#Helper function to format color identity
def format_color_identity(colors):
    color_map = {'W': 'White', 'U': 'Blue', 'B': 'Black', 'R': 'Red', 'G': 'Green'}
    return ', '.join([color_map.get(color, color) for color in colors]) if colors else 'Colorless'

In [ ]:
#Helper function to format mana cost with improved readability
def format_mana_cost(mana_cost):
    if not mana_cost:
        return 'None'

    #Define replacements for each symbol type
    color_symbols = {
        'W': 'White', 'U': 'Blue', 'B': 'Black', 'R': 'Red', 'G': 'Green',
        'C': 'Colorless', 'X': 'Variable'
    }

    #For hybrid mana symbols like {U/G} or {B/R}
    hybrid_symbols = {
        'W/U': 'White/Blue', 'U/B': 'Blue/Black', 'B/R': 'Black/Red', 'R/G': 'Red/Green',
        'G/W': 'Green/White', 'W/B': 'White/Black', 'U/R': 'Blue/Red', 'B/G': 'Black/Green',
        'R/W': 'Red/White', 'G/U': 'Green/Blue'
    }

    #Split mana costs like "{3}{U}{G}" to ["3", "U", "G"]
    symbols = mana_cost.replace('{', '').replace('}', ' ').split()
    formatted_cost = []

    for symbol in symbols:
        #Check if the symbol is a number (colorless mana)
        if symbol.isdigit():
            formatted_cost.append(symbol + " Colorless")
        #Check if the symbol is in the hybrid symbols dictionary
        elif symbol in hybrid_symbols:
            formatted_cost.append(hybrid_symbols[symbol])
        #Check if the symbol is in the color symbols dictionary
        elif symbol in color_symbols:
            formatted_cost.append(color_symbols[symbol])
        #Handle "split" cards by recognizing // as a divider
        elif symbol == '//':
            formatted_cost.append('//')
        else:
            formatted_cost.append(symbol)  #Unknown symbols

    #Join the formatted parts into a readable string
    return ' + '.join(formatted_cost)

In [ ]:
# Create a function to fetch cards from multiple sets using Scryfall's API
def fetch_cards_from_sets(set_codes):
    all_cards = []

    for set_code in set_codes:
        url = f"https://api.scryfall.com/cards/search?order=set&q=e%3A{set_code}&unique=prints"

        while url:
            response = requests.get(url)
            data = response.json()

            if 'data' in data:
                for card in data['data']:
                    #Use 'image_uris' if it exists and get the normal resolution; else, set as None
                    image_url = card.get('image_uris', {}).get('normal', None)

                    card_data = {
                        'Name': card.get('name', ''),
                        'Number': card.get('collector_number', ''),
                        'Type': card.get('type_line', ''),
                        'Subtype': ', '.join(card.get('subtypes', [])),
                        'Keywords': ', '.join(card.get('keywords', [])),
                        'Rarity': card.get('rarity', '').capitalize(),
                        'Color Identity': format_color_identity(card.get('color_identity', [])),
                        'Mana Cost': format_mana_cost(card.get('mana_cost', '')),
                        'Produced Mana': format_color_identity(card.get('produced_mana', [])),
                        'Power': card.get('power', ''),
                        'Toughness': card.get('toughness', ''),
                        'Defense': card.get('defense', ''),
                        'Hand Modifier': card.get('hand_modifier', ''),
                        'Life Modifier': card.get('life_modifier', ''),
                        'Commander Legal': 'Yes' if card.get('legalities', {}).get('commander') == 'legal' else 'No',
                        'EDHREC Rank': card.get('edhrec_rank', ''),
                        'Digital Availability': ', '.join(card.get('games', [])),
                        'Printed Text': card.get('oracle_text', ''),
                        'Set Code': card.get('set', ''),
                        'Set Name': card.get('set_name', ''),
                        'Release Date': card.get('released_at', ''),
                        'Flavor Text': card.get('flavor_text', ''),
                        'Image URL': image_url
                    }
                    all_cards.append(card_data)

                #Scryfall paginates results, so continue to the next page if exists
                url = data.get('next_page')
            else:
                url = None  #End the loop if no more data

    return all_cards


In [ ]:
#Function to create an Excel file from fetched card data with formatting
def create_excel_from_cards(cards_data, set_codes):
    #Ensure 'Set Code' is uppercase and 'Digital Availability' is title-cased
    for card in cards_data:
        if 'Set Code' in card:
            card['Set Code'] = card['Set Code'].upper()
        if 'Digital Availability' in card:
            card['Digital Availability'] = ' '.join(
                word.upper() if word.lower() == 'mtgo' else word.capitalize()
                for word in card['Digital Availability'].split()
            )

    #Clean up set codes to ensure they are non-empty, uppercase, and without extra spaces
    cleaned_set_codes = [code.strip().upper() for code in set_codes if code.strip()]

    #Build the file name from cleaned set codes
    file_name = f"scryfall_cards_{'_'.join(cleaned_set_codes)}.xlsx"

    #Write the data to an Excel file using pandas
    df = pd.DataFrame(cards_data)
    df = df.sort_values(by=['Name', 'Number']).reset_index(drop=True)
    df.to_excel(file_name, index=False, engine='openpyxl')

    #Load the workbook and select the active worksheet
    wb = load_workbook(file_name)
    ws = wb.active
    ws.freeze_panes = 'A2'

    #Remove columns that are entirely blank
    for col in list(ws.columns)[::-1]:  #Iterate in reverse to avoid data shifting issues with deleted column
        col_letter = get_column_letter(col[0].column)
        #Check if all cells in the column (except header) are empty
        if all(cell.value is None for cell in col[1:]):
            ws.delete_cols(col[0].column)

    #Define formatting for the header row
    header_font = Font(bold=True)
    header_fill = PatternFill(start_color="D3D3D3", end_color="D3D3D3", fill_type="solid")
    header_border = Border(
        left=Side(style='thin'), right=Side(style='thin'),
        top=Side(style='thin'), bottom=Side(style='thin')
    )

    #Apply header formatting with borders on all sides
    for cell in ws[1]:
        cell.font = header_font
        cell.alignment = Alignment(horizontal='center', vertical='center')
        cell.fill = header_fill
        cell.border = header_border

    #Set specific column widths
    fixed_15px_column = "Number"
    fixed_30px_columns = ["Name", "Type", "Subtype", "Keywords", "Rarity", "Color Identity", "Mana Cost", "Produced Mana"]
    fixed_40px_columns = ["Set Name"]
    long_text_columns = ["Printed Text", "Flavor Text"]
    image_url_column = "Image URL"

    #Apply column widths and formats
    for col in ws.columns:
        col_letter = get_column_letter(col[0].column)
        column_name = col[0].value

        if column_name == fixed_15px_column:
            ws.column_dimensions[col_letter].width = 15
            for cell in col[1:]:
                cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
                cell.number_format = '0'
        elif column_name in fixed_30px_columns:
            ws.column_dimensions[col_letter].width = 30
            for cell in col[1:]:
                cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
        elif column_name in fixed_40px_columns:
            ws.column_dimensions[col_letter].width = 40
            for cell in col[1:]:
                cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
        elif column_name in long_text_columns:
            ws.column_dimensions[col_letter].width = 35
            for cell in col[1:]:
                cell.alignment = Alignment(horizontal='left', vertical='center', wrap_text=True)
        elif column_name == image_url_column:
            for cell in col[1:]:
                if cell.value:
                    cell.hyperlink = cell.value
                    cell.value = "Link"
                    cell.font = Font(color="0000FF", underline="single")
                    cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
        else:
            #Auto-fit width for other columns
            max_length = max(len(str(cell.value)) for cell in col if cell.value) if col[1:] else 10
            adjusted_width = (max_length + 2) * 1.2
            ws.column_dimensions[col_letter].width = adjusted_width
            for cell in col[1:]:
                cell.alignment = Alignment(horizontal='center', vertical='center')

    #Format the EDHREC Rank column with comma separation
    for col in ws.columns:
        if col[0].value == "EDHREC Rank":
            for cell in col[1:]:
                cell.number_format = '#,##0'

    #Conditional formatting to turn rows red if Commander Legal is "No"
    ws.auto_filter.ref = ws.dimensions
    commander_legal_column = None
    for col in ws.iter_cols(1, ws.max_column):
        if col[0].value == "Commander Legal":
            commander_legal_column = col[0].column_letter
            break

    if commander_legal_column:
        row_range = f'A2:{get_column_letter(ws.max_column)}{ws.max_row}'
        rule = FormulaRule(formula=[f'${commander_legal_column}2="No"'], font=Font(color="FF0000"))
        ws.conditional_formatting.add(row_range, rule)

    #Save the workbook with formatting
    wb.save(file_name)
    print(f"Excel file '{file_name}' created successfully.")

In [ ]:
def main():
    set_codes_input = input("Enter one or more set codes (e.g., 'eld, m21' for Throne of Eldraine and Core Set 2021): ").strip()

    #Normalize and split set codes, handling common separator variations
    set_codes = [code.strip().upper() for code in set_codes_input.replace(" , ", ",").replace(", ", ",").replace(" ,", ",").split(",")]

    cards_data = fetch_cards_from_sets(set_codes)  #Use updated fetch function for multiple sets

    if cards_data:
        #Extract unique set names from the fetched card data
        set_names = set(card.get('Set Name', 'Unknown Set') for card in cards_data)

        #Format set names for display, adding a new line between names if there are multiple sets
        formatted_set_names = "\n".join(set_names)
        print(f"You've selected:\n{formatted_set_names}")

        #Pass set_codes as a list to create_excel_from_cards
        create_excel_from_cards(cards_data, set_codes)
    else:
        print("No cards found for the specified set codes. Please check the set codes and try again.")


In [ ]:
#Run the main function
if __name__ == '__main__':
    main()

Enter one or more set codes (e.g., 'eld, m21' for Throne of Eldraine and Core Set 2021): FDN
You've selected:
Foundations
Excel file 'scryfall_cards_FDN.xlsx' created successfully.
